# Trích xuất dữ liệu 50 trang đầu tiên theo từ khóa "Macbook Pro 14 M1 Pro" trên website Chợ Tốt

In [76]:
!pip install selenium


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


# Giới thiệu

Notebook này được sử dụng để trích xuất dữ liệu từ website Chợ Tốt liên quan đến sản phẩm "Macbook Pro 14 M1 Pro". Mục tiêu là thu thập thông tin chi tiết về các sản phẩm được rao bán, bao gồm:

- Tên sản phẩm
- Giá
- Quận
- Ngày lấy tin
- Kiểu sản phẩm
- Thời gian từ lúc trích xuất dữ liệu
- Link hình ảnh
- Link sản phẩm
- Trang số

Dữ liệu được thu thập từ 50 trang đầu tiên của kết quả tìm kiếm và được lưu trữ dưới dạng file CSV để dễ dàng phân tích và xử lý sau này. Công cụ chính được sử dụng bao gồm `Selenium` để tự động hóa việc thu thập dữ liệu và `BeautifulSoup` để phân tích HTML.

In [77]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [78]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Chạy ở chế độ không hiển thị giao diện
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)


In [79]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Định nghĩa một lớp để lưu trữ thông tin sản phẩm
# Những thuộc tinh cần lấy
# - Tên sản phẩm
# - Giá
# - Quận
# - Ngày lấy tin
# - Kiểu sản phẩm
# - Cách bao nhiêu thời gian từ lúc ngày lấy tin
# - link hình ảnh
# - Link sản phẩm
# - Trang số bao nhiêu

class Product:
    def __init__(self, name='', price='', district='', date_crawl='', product_type='', time_since_date_crawl='', image_link='', product_link='', page=1):
        self.name = name
        self.price = price
        self.district = district
        self.date_crawl = date_crawl
        self.product_type = product_type
        self.time_since_date_crawl = time_since_date_crawl
        self.image_link = image_link
        self.product_link = product_link
        self.page = page

    def __repr__(self):
        return f"Product(name={self.name}, price={self.price}, district={self.district}, date_posted={self.date_crawl}, product_type={self.product_type}, time_since_date_crawl={self.time_since_date_crawl}, image_link={self.image_link}, product_link={self.product_link}, page={self.page})"

In [ ]:
# loại sản phẩm được in theo dạng kiểu "16 GB . 512 GB", "32 GB . 512 GB" ,"32 GB . 1 TB . SSD"
# Vì dữ liệu phân loại nên mình sẽ thủ thập và nén thành một chuỗi rồi xử lý sau
def get_product_info(item):
    product_type = ''
    div = item.find('div', class_='c16i3beq r9vw5if')
    if div:  # Nếu div không rỗng
        # Tìm tất cả các thẻ span bên trong div
        for span in div.find_all("span"):
            if span.text != '': 
                product_type += span.text + ' | '
    else:
        product_type = "Unknown"  # Những sản phẩm không có thông tin sẽ được gán là Unknown
    return product_type


In [ ]:

from datetime import datetime
from bs4 import BeautifulSoup
from time import sleep

# Tạo một danh sách để lưu trữ các sản phẩm
macbooks = []

# Lặp qua các trang từ 1 đến 50
for pageIndex in range(1,50):
    # Mở trang web
    driver.get(f"https://www.chotot.com/mua-ban-tp-ho-chi-minh?q=macbook%20pro%2014%20m1&page={pageIndex}")

    # Lấy mã nguồn HTML của trang
    html = driver.page_source

    # Dùng BeautifulSoup để phân tích mã nguồn HTML
    soup = BeautifulSoup(html, "html.parser")

    # Tìm tất cả các thẻ <a> có class là "c14axl8t" (Vì đây là class của sản phẩm)
    items = soup.find_all("a", class_="c14axl8t")
    for item in items:
        # Tạo một đối tượng Product mới
        macbook = Product()

        # Lấy thông tin từ các thẻ con của thẻ <a>
        
        # Tên sản phẩm
        macbook.name = item.find('h3', class_ = 'adonovt').text

        # Giá sản phẩm
        macbook.price = item.find('span', class_ = 'bfe6oav tqqmhlc').text
        
        # Quận
        macbook.district = item.find_all('span', class_='c1u6gyxh')[0].text
        
        # Ngày lấy tin
        macbook.date_crawl = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Kiểu sản phẩm
        macbook.product_type = get_product_info(item)
        
        # Cách bao nhiêu thời gian từ lúc ngày lấy tin
        macbook.time_since_date_crawl = item.find_all('span', class_='c1u6gyxh')[1].text

        # Link hình ảnh
        image_link = item.find('div', class_ = 'a5f6zkj').find('span').find('noscript').find('img')
        if image_link and image_link['src'] != '':
            macbook.image_link = image_link['src']
        else: 
            continue # Nếu không có link hình ảnh thì bỏ qua sản phẩm này (Sản phẩm không có hình ảnh hoặc là quảng cáo)

        # Link sản phẩm
        macbook.product_link = 'https://www.chotot.com' + item['href']
        
        # Trang số bao nhiêu
        macbook.page = pageIndex

        # Thêm sản phẩm vào danh sách
        macbooks.append(macbook)

    # Không làm ảnh hưởng hệ thống nên mình sẽ nghỉ một chút
    # Đợi một chút trước khi tải trang tiếp theo (để tránh bị chặn IP)
    # Ở đây mình cho là 3 giây, bạn có thể điều chỉnh theo ý muốn
    sleep(3)



<span style="box-sizing:border-box;display:block;overflow:hidden;width:initial;height:initial;background:none;opacity:1;border:0;margin:0;padding:0;position:absolute;top:0;left:0;bottom:0;right:0"><noscript><img alt="" data-nimg="fill" decoding="async" loading="lazy" src="https://cdn.chotot.com/9lpAwUMGCmGQSgynQRlQZRmZis1L76xWJf1ZVb5tF4E/preset:listing/plain/62596f389647f6348a93041df23eab27-2929020700500316248.jpg" style="position:absolute;top:0;left:0;bottom:0;right:0;box-sizing:border-box;padding:0;border:none;margin:auto;display:block;width:0;height:0;min-width:100%;max-width:100%;min-height:100%;max-height:100%"/></noscript></span>
<span style="box-sizing:border-box;display:block;overflow:hidden;width:initial;height:initial;background:none;opacity:1;border:0;margin:0;padding:0;position:absolute;top:0;left:0;bottom:0;right:0"><noscript><img alt="" data-nimg="fill" decoding="async" loading="lazy" src="https://cdn.chotot.com/L5c3e-nN58HNyKWuUC9gBclFKZMaecD9AMyh5Gql5Fs/preset:listing/p

In [83]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd

# Chuyển đổi danh sách sản phẩm thành DataFrame
df = pd.DataFrame([macbook.__dict__ for macbook in macbooks])

# Lưu DataFrame vào file CSV
df.to_csv('macbooks.csv', index=False, encoding='utf-8-sig')

# Đóng trình duyệt
driver.quit()